Testing of 3 models
-------
Evaluates A0, A1 and A2 models on the test dataset.

Note:
  - Check the paths to models and dataset.

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!unzip -q "/content/drive/My Drive/DIPLOMSKI/oxford-iiit/test.zip" -d '/content' 


import torch
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import json
import copy

BATCH_SIZE = 64
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

NUM_CLASSES = 37
INPUT_SIZE = 224
DATA_DIR = '/content/'

#Data preparation
data_transforms = {
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(INPUT_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), 
                                          data_transforms[x]) for x in ['test']}
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                                    batch_size=BATCH_SIZE, 
                                                    shuffle=False) for x in ['test']}


#Reshape last layer of the model
model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=False)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, NUM_CLASSES)

PREDS = []
TRUE = []
for PREFIX in ['A0', 'A1', 'A2']:

  #Build the model and move it to GPU (if available)
  SAVE_PATH = f'/content/drive/My Drive/DIPLOMSKI/{PREFIX}'
  VAL_MODEL_NAME = f'{PREFIX}_best_val_model.pt'
  
  #load the saved parameters into the model
  model.load_state_dict(torch.load(f'{SAVE_PATH}/{VAL_MODEL_NAME}'))
  if DEVICE == torch.device('cuda:0'):
    model.cuda()

  criterion = nn.CrossEntropyLoss()
  if DEVICE == torch.device('cuda:0'):
    criterion.cuda()

  dataloaders = dataloaders_dict

  #not updating the parameters - evaluation mode
  model.eval()

  test_acc = 0.0
  P = []; T = []
  for inputs, labels in dataloaders['test']:
    #moving the dataset to the DEVICE
    inputs = inputs.to(DEVICE); labels = labels.to(DEVICE)

    #disable gradient propagation
    with torch.set_grad_enabled(False):

      outputs = model(inputs); loss = criterion(outputs, labels)

      _, preds = torch.max(outputs, 1)

      P.append(outputs)
      T.append(labels.data)
    # statistics
    test_acc += torch.sum(preds == labels.data)
    
  PREDS.append(np.array(P))
  TRUE.append(np.array(T))
  test_acc = (test_acc / len(dataloaders['test'].dataset)).item()
    
  print(f"Model {PREFIX}: Test Acc: {test_acc}")


Mounted at /content/drive


Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip


Model A0: Test Acc: 0.7096556425094604
Model A1: Test Acc: 0.8440243005752563
Model A2: Test Acc: 0.8575286865234375
